<a href="https://colab.research.google.com/github/Iddo77/YOLO_drone_detection/blob/main/YOLO_drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-learn

  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [2]:
import os
import random
from sklearn.model_selection import train_test_split

In [7]:
# constants
NUM_TRAIN = 300
RANDOM_STATE = 36
# replace with correct root of project 
PROJECT_ROOT = r"D:\OU Master AI\Research methods\Project"

In [8]:
# get the path to our own project
project_path = os.path.join(PROJECT_ROOT, 'YOLO_drone_detection')
print(project_path)
# get a the path to the dataset folder
dataset_path = os.path.join(project_path, "dataset")
print(dataset_path)
# get the path to the folder with the YOLO v7 code
yolo_path = os.path.join(PROJECT_ROOT, "yolov7")
print(yolo_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection
D:\OU Master AI\Research methods\Project\YOLO_drone_detection\dataset
D:\OU Master AI\Research methods\Project\yolov7


In [9]:
def create_train_val_test_txt_files():
    # get all image files and shuffle them
    all_images = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    random.shuffle(all_images)

    # Split the dataset into training, validation, and test sets
    train_images = all_images[:NUM_TRAIN]
    test_val_images = all_images[NUM_TRAIN:]

    # split the test_val set into validation and test sets
    val_images, test_images = train_test_split(test_val_images, 
                                               train_size=0.5, 
                                               test_size=0.5, 
                                               random_state=RANDOM_STATE)

    # Write the paths to the train, val, and test text files
    with open(os.path.join(project_path, 'train.txt'), 'w') as f:
        for img in train_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'val.txt'), 'w') as f:
        for img in val_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'test.txt'), 'w') as f:
        for img in test_images:
            f.write(os.path.join(dataset_path, img) + '\n')

In [10]:
def create_custom_yaml_file():
    """
    Creates a custom YAML file for YOLO that specifies the paths to the training, 
    validation, and test sets.
    """
    yaml_text = f"""train: '{os.path.join(project_path, "train.txt")}'
val: '{os.path.join(project_path, "val.txt")}'
test: '{os.path.join(project_path, "test.txt")}'
nc: 1
names: ['drone']
"""
    with open(os.path.join(project_path, 'custom.yaml'), 'w') as f:
        f.write(yaml_text)


In [11]:
create_train_val_test_txt_files()
create_custom_yaml_file()

In [13]:
# finetune the base model using the custom.yaml file created above
os.chdir(yolo_path)
!python train.py --workers 8 --device 0 --batch-size 8 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

wandb: Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)

autoanchor: Analyzing anchors... anchors/target = 3.30, Best Possible Recall (BPR) = 1.0000
                 all         529         579     0.00367       0.722      0.0083     0.00105
                 all         529         579     0.00923       0.617     0.00697    0.000859
                 all         529         579     0.00365      0.0759     0.00746    0.000991
                 all         529         579     0.00898      0.0345     0.00937     0.00264
                 all         529         579     0.00684       0.325     0.00455    0.000686
                 all         529         579       0.011      0.0691     0.00541    0.000792
                 all         529         579      0.0265      0.0725     0.00871     0.00129
                 all         529         579     0.00315       0.472     0.00245    0.000368
                 all         529         579     0.00507        0.25     0

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

